# Extracting symbols from image using `OpenCV`

In [1]:
import os
import pickle
import cv2
import numpy as np
from functools import cmp_to_key
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf

from tensorflow import keras
%matplotlib inline 
import matplotlib.pyplot as plt

Note that these functions have small logic differences <br> 
between them and the functions in `imgPreProcess.py` <br>
This is for debugging purposes

In [2]:
def extractSymbols(imgOrig, showSteps = False, verticalSymbols = False):
    debugImgSteps = []
    imgGray = cv2.cvtColor(imgOrig,cv2.COLOR_BGR2GRAY)
    imgFiltered = cv2.medianBlur(imgGray, 5)
    debugImgSteps.append(imgFiltered)
    
    imgCanny = cv2.Canny(imgFiltered, 50,180)
    debugImgSteps.append(imgCanny)

    kernel = np.ones((5,5), np.uint8)
    imgDilated = cv2.dilate(imgCanny, kernel, iterations=5)
    debugImgSteps.append(imgDilated)

    contours, _= cv2.findContours(imgDilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    boundingBoxes = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        boundingBoxes.append((x,y,w,h))

    global rowsG
    rowsG, _, _ = imgOrig.shape
    key_leftRightTopBottom = cmp_to_key(leftRightTopBottom)
    if (verticalSymbols):
        boundingBoxes = sorted(boundingBoxes, key=key_leftRightTopBottom)
    else:
        boundingBoxes = sorted(boundingBoxes, key=lambda x : x[0])

    symbols = []
    for box in boundingBoxes:
        x,y,w,h = box
        mathSymbol = imgOrig[y:y+h, x:x+w]
        mathSymbol = cv2.cvtColor(mathSymbol, cv2.COLOR_BGR2GRAY) #converting to Gray as tensorflow deals with grayscale or RGB, not BGR
        mathSymbol = cv2.resize(mathSymbol, (45,45), interpolation=cv2.INTER_AREA) #to have the same size as trained images in the dataset
        debugImgSteps.append(mathSymbol)
        mathSymbolF = mathSymbol.astype('float32') #optional: tensorflows deals with float32, not uint8
        symbols.append(mathSymbolF)

    if showSteps:
        dispImages(debugImgSteps)

    return symbols
        

In [3]:
def leftRightTopBottom(tup1, tup2):
    x1, y1, _, _ = tup1
    x2, y2, _, _ = tup2
    rows = rowsG
    yRegion1, yRegion2 = -1, -1

    for i in range(4):
        if y1 < rows/4 + rows*(i/4.0):
            yRegion1 = i
            break
    else:
        if yRegion1 == -1:
            yRegion1 = 4

    for i in range(4):
        if y2 < rows/4 + rows*(i/4.0):
            yRegion2 = i
            break
    else:
        if yRegion2 == -1:
            yRegion2 = 4
    
    if yRegion1 < yRegion2:
        return -1
    elif yRegion2 < yRegion1:
        return 1
    elif x1 <= x2:
        return -1
    else:
        return 1

In [4]:
def dispImages(imgs):
    for img in imgs:
        cv2.imshow('Image', img)
        cv2.waitKey(0)
    else:
        cv2.destroyAllWindows()

In [37]:
img = cv2.imread('tests/test1.png')
symbols = extractSymbols(img, showSteps=True)

# Creating dictionary that maps folder names to latex

example of folder names in "mathSymbolsDataset": <br>
<img src="guideImages/datasetFolders.png" width=400 height=300>

using `r` to make the string `raw` to avoid confusing strings like `\n` with python's new line <br>
however, the values will now have two backslashes (e.g. `\\n`), thus, we will later need to replace each `\\` with `\`

In [6]:
dic = {
    "-": r"-",
    "(": r"(",
    ")": r")",   
    "+": r"+",
    "=": r"=",
    "0": r"0",
    "1": r"1",
    "2": r"2",
    "3": r"3",
    "4": r"4",
    "5": r"5",
    "6": r"6",
    "7": r"7",
    "8": r"8",
    "9": r"9",
    "geq": r"\geq",
    "gt": r">",
    "i": r"i",
    "in": r"\in",
    "int": r"\int",
    "j": r"j",
    "leq": r"\le",
    "lt": r"<",
    "neq": r"\neq",
    "pi": r"\Pi",
    "sum": r"\sum",
    "theta": r"\theta",
    "times": r"\times",
    "w": r"w",
    "X": r"\X",
    "y": r"y",
    "z": r"z"
}

# Preparing the dataset

## Reading the kaggle [dataset](https://www.kaggle.com/datasets/xainano/handwrittenmathsymbols?resource=download)

Steps:
1. create a list of images and another list of labels for each image
2. store them in pickle files for easy retrieval when re-running the code 

In [7]:
def loadData(dataDir):
    imgs = []
    labels = []
    for key, value in dic.items():
        path = os.path.join(dataDir, key)
        for imgName in os.listdir(path):
            try:
                img = cv2.imread(os.path.join(path, imgName), cv2.COLOR_BGR2GRAY) 
                imgs.append(img)
                labels.append(value)
            except Exception as e:
                print(e)    
    return (imgs, labels)

The following cell is commented as it takes a long time (10min if image RGB, 1min otherwise) to create the pickle files

In [10]:
#imgs, labels = loadData('mathSymbolsDataset/')
#with open("x_symbols.pickle", 'wb') as f:
#    pickle.dump(imgs, f)
#with open("y_latex.pickle", 'wb') as f:
#    pickle.dump(labels, f)

In [8]:
with open("x_symbols_reduced.pickle", 'rb') as f:
    imgs = pickle.load(f)
with open("y_latex_reduced.pickle", 'rb') as f:
    labels = pickle.load(f)

## converting text labels (latex) to numeric codes

In [9]:
latexToNums = {k: v for v, k in enumerate(np.unique(labels))}
#this dictionary is to revert the predicted numeric code back to latex: 
numsToLatex = {v: k for v, k in enumerate(np.unique(labels))}
latexToNums

{'(': 0,
 ')': 1,
 '+': 2,
 '-': 3,
 '0': 4,
 '1': 5,
 '2': 6,
 '3': 7,
 '4': 8,
 '5': 9,
 '6': 10,
 '7': 11,
 '8': 12,
 '9': 13,
 '<': 14,
 '=': 15,
 '>': 16,
 '\\Pi': 17,
 '\\X': 18,
 '\\geq': 19,
 '\\in': 20,
 '\\int': 21,
 '\\le': 22,
 '\\neq': 23,
 '\\sum': 24,
 '\\theta': 25,
 '\\times': 26,
 'i': 27,
 'j': 28,
 'w': 29,
 'y': 30,
 'z': 31}

In [10]:
labelsNums = [latexToNums[label] for label in labels]

## Splitting the data into train and test data

Note that `stratify` is used to split the dataset into train and test sets <br> 
in a way that preserves the same proportions of examples in each class as observed in the original dataset <br>
[(source)](https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/#:~:text=is%20desirable%20to-,split%20the%20dataset%20into,stratified%20train-test%20split.,-We%20can%20achieve)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.33, stratify=labels, random_state=42)

## Normalizing image pixels

In [12]:
x_train = tf.keras.utils.normalize(x_train, axis=1) #similar to dividing by 255 (but not equivalent in result)
x_test = tf.keras.utils.normalize(x_test, axis=1) #Also, don't know why we are using "axis=1" specifically, but that's what's normally used with image normalization

### Converting `y` labels to numeric codes instead of strings
Because `keras` models accept numbers not strings

In [13]:
y_train_nums = [latexToNums[latex] for latex in y_train]
y_test_nums = [latexToNums[latex] for latex in y_test]
y_train_nums

[10,
 18,
 2,
 4,
 3,
 4,
 31,
 6,
 3,
 2,
 3,
 15,
 18,
 2,
 6,
 3,
 7,
 3,
 1,
 8,
 6,
 3,
 3,
 6,
 25,
 10,
 15,
 18,
 2,
 11,
 0,
 6,
 7,
 7,
 18,
 15,
 7,
 5,
 12,
 15,
 8,
 15,
 1,
 19,
 7,
 18,
 2,
 4,
 4,
 3,
 3,
 2,
 3,
 3,
 5,
 12,
 28,
 1,
 15,
 8,
 3,
 0,
 8,
 5,
 3,
 30,
 27,
 6,
 2,
 3,
 1,
 6,
 6,
 18,
 5,
 6,
 7,
 18,
 18,
 3,
 4,
 27,
 26,
 7,
 18,
 3,
 15,
 6,
 3,
 18,
 1,
 5,
 15,
 4,
 18,
 4,
 18,
 1,
 3,
 3,
 1,
 15,
 18,
 17,
 2,
 4,
 2,
 0,
 13,
 3,
 19,
 1,
 0,
 0,
 27,
 13,
 7,
 31,
 30,
 18,
 3,
 23,
 18,
 5,
 17,
 2,
 13,
 7,
 5,
 1,
 2,
 1,
 6,
 5,
 5,
 3,
 26,
 3,
 5,
 1,
 7,
 1,
 5,
 7,
 5,
 16,
 18,
 7,
 2,
 3,
 0,
 2,
 5,
 6,
 5,
 22,
 3,
 21,
 2,
 31,
 8,
 2,
 2,
 18,
 0,
 6,
 6,
 21,
 3,
 3,
 3,
 5,
 5,
 3,
 19,
 18,
 26,
 13,
 26,
 5,
 3,
 6,
 5,
 7,
 2,
 31,
 24,
 0,
 3,
 27,
 30,
 18,
 30,
 5,
 7,
 11,
 18,
 24,
 6,
 5,
 18,
 3,
 6,
 2,
 31,
 3,
 2,
 9,
 5,
 4,
 3,
 0,
 30,
 9,
 30,
 17,
 3,
 18,
 27,
 0,
 0,
 24,
 26,
 12,
 6,
 5,
 28,
 5,
 28,
 5,

### Making sure all datasets are `ndarray` not `list`
Because `keras` models accept `ndarray`

In [14]:
type(x_train), type(x_test), type(y_train_nums), type(y_test_nums)

(numpy.ndarray, numpy.ndarray, list, list)

In [15]:
y_train_nums = np.array(y_train_nums)
y_test_nums = np.array(y_test_nums)

In [16]:
type(y_train_nums), type(y_test_nums)

(numpy.ndarray, numpy.ndarray)

## Working Model

The performance measure here is the accuracy metric the NN model is. <br>
Note that the loss function and metrics arguments in `model.compile` are considered the model's utility (evaluation) function <br>
as they tell the model how effective the training on the dataset is.

In [17]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2025, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2025, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.softmax))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [19]:
model.fit(x_train, y_train_nums, epochs=5)

Epoch 1/5
5465/5465 [==============================] - 282s 51ms/step - loss: 1.4541 - accuracy: 0.5773
Epoch 2/5
5465/5465 [==============================] - 281s 51ms/step - loss: 0.8027 - accuracy: 0.7626
Epoch 3/5
5465/5465 [==============================] - 288s 53ms/step - loss: 0.6237 - accuracy: 0.8137
Epoch 4/5
5465/5465 [==============================] - 286s 52ms/step - loss: 0.5004 - accuracy: 0.8504
Epoch 5/5
5465/5465 [==============================] - 268s 49ms/step - loss: 0.4243 - accuracy: 0.8717


In [20]:
model.save("ThennModel")

INFO:tensorflow:Assets written to: ThennModel\assets


### Load the Model from Here:

In [ ]:
model = keras.models.load_model("ThennModel")

### Predicting the test case

Earlier we had the image segmented and stored the symbols in the variable symbols so we will predict for each symbol and see the result

In [38]:
print(np.argmax(model.predict(symbols[0].reshape(1,45,45))))

18


Looking in the dictionary above, 18 matches to x which is correct

In [39]:
print(np.argmax(model.predict(symbols[1].reshape(1,45,45))))

2


Looking in the dictionary above, 2 matches to + which is correct

In [40]:
print(np.argmax(model.predict(symbols[2].reshape(1,45,45))))

30


Looking in the dictionary above, 30 matches to y which is correct

In [41]:
print(np.argmax(model.predict(symbols[3].reshape(1,45,45))))

15


Looking in the dictionary above, 15 matches to = which is correct

In [42]:
print(np.argmax(model.predict(symbols[4].reshape(1,45,45))))

4


Looking in the dictionary above, 4 matches to 0 which is correct

Note that the sequence of the symbols is in the sequence of the image segmentation.

## Experimenting with Different Neural Networks

### First Attempt:

### 1. Sequential vs Functional models

* Sequential is a linear stack of layers. In other words, the layer `i` is connected only to layers `i-1` and `i+1`
* Functional is more dynamic, as each layer can connect to any other layer in the neural network

Since the images are small in size, and the problem is relatively simple, we'll use a sequential model

In [19]:
model = tf.keras.models.Sequential()

### 2. Model layers

In [20]:
# for easier processing: flatten image (e.g. 45x45 will become 1x2025)
model.add(tf.keras.layers.Flatten())
# 128 nodes are chosen as they are a power of 2 (2^7) which makes computation easier, and the images are not large (45x45) so 128 nodes should suffice
# relu is the default activation function to use
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
# add another layer because if you have one, then you're getting linear relations only between the image's features, while two layers makes it non-linear
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
# number of classifications == number of stored latex strings == len(latexToNums) == 79
# using softmax as it converts the scores to a normalized probability distribution
model.add(tf.keras.layers.Dense(len(latexToNums), activation=tf.nn.softmax))

### 3. Model compilation

In [21]:
# "compiling" means passing the settings for actually optimizing/training the model we've defined
model.compile(optimizer='adam', # same logic as relu, great default optimizer to start with
              loss='sparse_categorical_crossentropy', # A neural network doesn't actually attempt to maximize accuracy. It attempts to minimize loss, this loss function is also a great default
              metrics=['accuracy']) # ratio between the number of correct predictions to the total number of predictions.

### 4. Model training

"A good rule of thumb is to start with a value that is 3 times the number of columns in your data." <br>
[(source)](https://gretel.ai/gretel-synthetics-faqs/how-many-epochs-should-i-train-my-model-with) <br>
Therefore, we start by with 45*3 = 135 epochs (i.e. number of passes of the entire training dataset the machine learning algorithm has completed)

In [22]:
#model.fit(x_train, y_train_nums, epochs=135)

### 5. Save the model for later use

Technical note: pickle doesn't save models correctly, as it outputs this error when loading the pickle file: <br><br>
FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://0eb44777-6983-466e-ac15-adfa9d3dae07/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'. <br><br>
 That's why we are using keras's `save()` and `load_model()`

In [23]:
#model.save("nnModel")

### 6. Load the model

In [6]:
model = keras.models.load_model("nnModel")

In [51]:
symbols[0].reshape(1,45,45).shape

(1, 45, 45)

In [119]:
symTest = symbols[8]

In [120]:
print(np.argmax(model.predict(symTest.reshape(1,45,45))))

11


This model wasn't accurate with the results so we tried to work with convolutional Neural Networks.

The convolutional neural network was seemingly a good solution at the time since it applies filters on the images to extract its features, but the problem is handwritten characters barely had features so it still wasn't accurate, and it caused some overfitting.

### Second Attempt

In [ ]:
from numpy import unique, argmax
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from keras.utils.vis_utils  import plot_model
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation  ='relu', input_shape=(45,45,1)))
model.add(MaxPool2D((2,2))) 
#batch normalization, try averagepool
#leak
model.add(Conv2D(48, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2025, activation='relu'))
model.add(Dense(79, activation='softmax'))

In [ ]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
x = model.fit(x_train, y_train_nums, epochs=10, batch_size=128, verbose=2, validation_split=0.1)

Epoch 1/10
1722/1722 - 386s - loss: 0.6359 - accuracy: 0.8275 - val_loss: 0.2793 - val_accuracy: 0.9140 - 386s/epoch - 224ms/step
Epoch 2/10
1722/1722 - 418s - loss: 0.2386 - accuracy: 0.9268 - val_loss: 0.1760 - val_accuracy: 0.9441 - 418s/epoch - 243ms/step
Epoch 3/10
1722/1722 - 401s - loss: 0.1689 - accuracy: 0.9461 - val_loss: 0.1280 - val_accuracy: 0.9589 - 401s/epoch - 233ms/step
Epoch 4/10
1722/1722 - 423s - loss: 0.1312 - accuracy: 0.9569 - val_loss: 0.1148 - val_accuracy: 0.9613 - 423s/epoch - 246ms/step
Epoch 5/10
1722/1722 - 406s - loss: 0.1091 - accuracy: 0.9634 - val_loss: 0.0922 - val_accuracy: 0.9711 - 406s/epoch - 236ms/step
Epoch 6/10
1722/1722 - 410s - loss: 0.0934 - accuracy: 0.9684 - val_loss: 0.0835 - val_accuracy: 0.9761 - 410s/epoch - 238ms/step
Epoch 7/10
1722/1722 - 428s - loss: 0.0828 - accuracy: 0.9721 - val_loss: 0.0740 - val_accuracy: 0.9781 - 428s/epoch - 248ms/step
Epoch 8/10
1722/1722 - 430s - loss: 0.0739 - accuracy: 0.9752 - val_loss: 0.0722 - val_acc

In [ ]:
#model.save("FModel")

In [ ]:
#model = keras.models.load_model("FModel")

### Third Attempt

Due to resources limitations, we couldn't run this model

In [ ]:
#x_train = np.array(x_train)

In [ ]:
#model = Sequential()
#model.add(Conv2D(64, (3,3), input_shape=x_train.shape))
#model.add(Activation("relu"))
#model.add(MaxPool2D(pool_size=(2,2)))

#model.add(Conv2D(64, (3,3)))
#model.add(Activation("relu"))
#model.add(MaxPool2D(pool_size=(2,2)))

#model.add(Flatten())
#model.add(Dense(64))
#model.add(Dense(1))
#model.add(Activation('sigmoid'))

#model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
#model.fit(x_train, y_train_nums, batch_size=128, validation_split=0.1)


### Fourth Attempt

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), activation  ='relu', input_shape=(45,45,1)))
model.add(MaxPool2D((2,2))) #batch normalization, try averagepool
#leak
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))

#model.add(Conv2D(64, (3,3), activation='relu'))
#model.add(MaxPool2D((2,2)))

model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(2025, activation='relu'))
#model.add(Dense(2025, activation='relu'))
BatchNormalization(axis=1)
model.add(Dense(79, activation='softmax'))

In [ ]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
x = model.fit(x_train, y_train_nums, epochs=10, batch_size=128, verbose=2, validation_split=0.1)

Epoch 1/10
1722/1722 - 536s - loss: 0.9281 - accuracy: 0.7486 - val_loss: 0.3709 - val_accuracy: 0.8863 - 536s/epoch - 311ms/step
Epoch 2/10
1722/1722 - 546s - loss: 0.4311 - accuracy: 0.8686 - val_loss: 0.2845 - val_accuracy: 0.9116 - 546s/epoch - 317ms/step
Epoch 3/10
1722/1722 - 517s - loss: 0.3508 - accuracy: 0.8897 - val_loss: 0.2320 - val_accuracy: 0.9252 - 517s/epoch - 300ms/step
Epoch 4/10
1722/1722 - 496s - loss: 0.3076 - accuracy: 0.9011 - val_loss: 0.2139 - val_accuracy: 0.9291 - 496s/epoch - 288ms/step
Epoch 5/10
1722/1722 - 531s - loss: 0.2768 - accuracy: 0.9098 - val_loss: 0.1982 - val_accuracy: 0.9345 - 531s/epoch - 309ms/step
Epoch 6/10
1722/1722 - 531s - loss: 0.2555 - accuracy: 0.9155 - val_loss: 0.1801 - val_accuracy: 0.9393 - 531s/epoch - 308ms/step
Epoch 7/10
1722/1722 - 535s - loss: 0.2383 - accuracy: 0.9213 - val_loss: 0.1676 - val_accuracy: 0.9434 - 535s/epoch - 311ms/step
Epoch 8/10
1722/1722 - 496s - loss: 0.2243 - accuracy: 0.9252 - val_loss: 0.1571 - val_acc

In [ ]:
p = model.predict(symbols[0].reshape(1,45,45))
print(argmax(p))

20


In [ ]:
model.save("F1Model")

INFO:tensorflow:Assets written to: F1Model\assets


### Fifth Attempt

Since we discovred that our dataset is imbalanced and it caused over domination by some classes, we tried to create class weights to weight each class and balance it, yet it did not work.

## Getting Class Weights

In [ ]:
numLabels, counts = np.unique(y_train, return_counts=True)
numLabelsToFreq = dict(zip(numLabels, counts))
numLabelsToFreq

{0: 9577,
 1: 9618,
 2: 16825,
 3: 22778,
 4: 4632,
 5: 17768,
 6: 17514,
 7: 7309,
 8: 4955,
 9: 2375,
 10: 2089,
 11: 1949,
 12: 2056,
 13: 2504,
 14: 320,
 15: 8780,
 16: 173,
 17: 1562,
 18: 17818,
 19: 464,
 20: 31,
 21: 1837,
 22: 652,
 23: 374,
 24: 1802,
 25: 1873,
 26: 2178,
 27: 3444,
 28: 1029,
 29: 373,
 30: 6258,
 31: 3933}

In [ ]:
maxlabelImgs = max(numLabelsToFreq.values())
labelWeights = {label : maxlabelImgs / float(numImgs) for label, numImgs in numLabelsToFreq.items()}
for k,v in labelWeights.items():
    if(v > 50):
        labelWeights[k] = 50.0
labelWeights

{0: 2.378406599143782,
 1: 2.368267831149927,
 2: 1.3538187221396731,
 3: 1.0,
 4: 4.917530224525043,
 5: 1.2819675821701937,
 6: 1.3005595523581135,
 7: 3.1164317964153785,
 8: 4.596972754793138,
 9: 9.590736842105263,
 10: 10.903781713738631,
 11: 11.687018984094408,
 12: 11.078793774319067,
 13: 9.09664536741214,
 14: 50.0,
 15: 2.5943052391799544,
 16: 50.0,
 17: 14.58258642765685,
 18: 1.2783701874508924,
 19: 49.09051724137931,
 20: 50.0,
 21: 12.399564507348938,
 22: 34.93558282208589,
 23: 50.0,
 24: 12.640399556048834,
 25: 12.161238654564869,
 26: 10.45821854912764,
 27: 6.6138211382113825,
 28: 22.13605442176871,
 29: 50.0,
 30: 3.639821029082774,
 31: 5.791507754894482}

### Sixth Attempt

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1000, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1000, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1000, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.softmax))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
5465/5465 [==============================] - 120s 22ms/step - loss: 1.5544 - accuracy: 0.5430
Epoch 2/10
5465/5465 [==============================] - 120s 22ms/step - loss: 0.8703 - accuracy: 0.7398
Epoch 3/10
5465/5465 [==============================] - 134s 25ms/step - loss: 0.6568 - accuracy: 0.8006
Epoch 4/10
5465/5465 [==============================] - 147s 27ms/step - loss: 0.5453 - accuracy: 0.8328
Epoch 5/10
5465/5465 [==============================] - 142s 26ms/step - loss: 0.4646 - accuracy: 0.8571
Epoch 6/10
5465/5465 [==============================] - 133s 24ms/step - loss: 0.4109 - accuracy: 0.8715
Epoch 7/10
5465/5465 [==============================] - 147s 27ms/step - loss: 0.3661 - accuracy: 0.8848
Epoch 8/10
5465/5465 [==============================] - 149s 27ms/step - loss: 0.3264 - accuracy: 0.8965
Epoch 9/10
5465/5465 [==============================] - 152s 28ms/step - loss: 0.2941 - accuracy: 0.9051
Epoch 10/10
5465/5465 [==============================] 

In [ ]:
#model.save("nnModel4")

INFO:tensorflow:Assets written to: nnModel4\assets


In [ ]:
#model = keras.models.load_model("nnModel4")